- Set up the Environment

In [1]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.split(os.getcwd())[0])


import time
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.metrics import r2_score

from tkat_loc import TKAT # this is the KAN version with Transformer

tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

from tkat_model_utils import generate_data_w_known_inputs

- Read sample financial data

In [2]:
df = pd.read_parquet('data.parquet')
df = df[(df.index >= pd.Timestamp('2020-01-01')) & (df.index < pd.Timestamp('2023-01-01'))]
# assets = ['BTC', 'ETH', 'ADA', 'XMR', 'EOS', 'MATIC', 'TRX', 'FTM', 'BNB', 'XLM', 'ENJ', 'CHZ', 'BUSD', 'ATOM', 'LINK', 'ETC', 'XRP', 'BCH', 'LTC']
assets = ['BTC']
df = df[[c for c in df.columns if 'quote asset volume' in c and any(asset in c for asset in assets)]]
df.columns = [c.replace(' quote asset volume', '') for c in df.columns]
known_input_df = pd.DataFrame(index=df.index, data=np.array([df.reset_index()['group'].apply(lambda x: (x.hour)).values, df.reset_index()['group'].apply(lambda x: (x.dayofweek)).values]).T, columns = ['hour', 'dayofweek'])
display(df)
display(known_input_df)

,BTC
group,
2020-01-01 00:00:00,3.675857e+06
2020-01-01 01:00:00,6.365953e+06
2020-01-01 02:00:00,4.736719e+06
2020-01-01 03:00:00,5.667367e+06
2020-01-01 04:00:00,3.379094e+06
...,...
2022-12-31 19:00:00,6.704605e+07
2022-12-31 20:00:00,4.344849e+07
2022-12-31 21:00:00,5.992803e+07


,hour,dayofweek
group,,
2020-01-01 00:00:00,0,2
2020-01-01 01:00:00,1,2
2020-01-01 02:00:00,2,2
2020-01-01 03:00:00,3,2
2020-01-01 04:00:00,4,2
...,...,...
2022-12-31 19:00:00,19,5
2022-12-31 20:00:00,20,5
2022-12-31 21:00:00,21,5


- Read last 1000 rows of data for quicker runtime

In [3]:
df = df.tail(1000)
known_input_df = known_input_df.tail(1000)

- Define model parameters

In [4]:
N_MAX_EPOCHS = 100
BATCH_SIZE = 128
early_stopping_callback = lambda : tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=6,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=6,
)
lr_callback = lambda : tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=3,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda : [early_stopping_callback(), lr_callback(), tf.keras.callbacks.TerminateOnNaN()]

In [5]:
num_hidden = 100
num_heads = 4
num_embedding = 1
n_ahead = 4
sequence_length = 5 * n_ahead

- Transform dataframe to correct format for TKAT

In [6]:
X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test = generate_data_w_known_inputs(
    df, 
    known_input_df, 
    sequence_length, 
    n_ahead
    )

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((509, 24, 3), (128, 24, 3), (509, 4), (128, 4))

- One-Hot encoding for classification

In [7]:
y_train = np.random.randint(0, n_ahead, y_train.shape[0])
y_test = np.random.randint(0, n_ahead, y_test.shape[0])

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(max_categories=n_ahead)
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()

In [8]:
num_unknow_features = len(assets)
num_know_features = X_train.shape[2] - num_unknow_features

- Compile TKAT model for classification

In [9]:
from keras.losses import CategoricalCrossentropy
model = TKAT(sequence_length, num_unknow_features, num_know_features, num_embedding, num_hidden, num_heads, n_ahead, use_tkan = True)

# model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy']) # from_logits, allow us to automatically perform softmax activation in the loss

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 24, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_layer     │ (None, 24, 1, 3)  │          0 │ input_layer[0][0] │
│ (EmbeddingLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ past_observed_and_… │ (None, 20, 1, 3)  │          0 │ embedding_layer[… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vsn_past_features   │ (None, 20, 100)   │    103,824 │ past_observed_an… │
│ (VariableSelection… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ future_known        │ (None, 4, 1, 2)   │          0 │ embedding_layer[… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder             │ [(None, 20, 100), │     71,612 │ vsn_past_feature… │
│ (RecurrentLayer)    │ (None, 100),      │            │                   │
│                     │ (None, 100),      │            │                   │
│                     │ (None, 100)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vsn_future_features │ (None, 4, 100)    │     72,614 │ future_known[0][… │
│ (VariableSelection… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, 4, 100)    │     71,612 │ vsn_future_featu… │
│ (RecurrentLayer)    │                   │            │ encoder[0][1],    │
│                     │                   │            │ encoder[0][2],    │
│                     │                   │            │ encoder[0][3]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 24, 100)   │          0 │ encoder[0][0],    │
│ (Concatenate)       │                   │            │ decoder[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gate_7 (Gate)       │ (None, 24, 100)   │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 24, 100)   │          0 │ vsn_past_feature… │
│ (Concatenate)       │                   │            │ vsn_future_featu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_and_norm_7      │ (None, 24, 100)   │        200 │ gate_7[0][0],     │
│ (AddAndNorm)        │                   │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ grn_7 (GRN)         │ (None, 24, 100)   │          0 │ add_and_norm_7[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 100)   │    161,300 │ grn_7[0][0],      │
│ (MultiHeadAttentio… │                   │            │ grn_7[0][0],      │
│                     │                   │            │ grn_7[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2400)      │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 4)         │      9,604 │ flatten[0][0]     │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 490,766 (1.87 MB)

 Trainable params: 490,746 (1.87 MB)

 Non-trainable params: 20 (80.00 B)

- Model Training

In [10]:
model.save('tkat_tf_model.keras')

In [11]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=10, validation_split=0.2, 
                    callbacks=callbacks(), 
                    shuffle=True, verbose = True)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 41s 900ms/step - accuracy: 0.2886 - loss: 1.4625 - val_accuracy: 0.2255 - val_loss: 1.5441 - learning_rate: 0.0010
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 0.2597 - loss: 1.4866 - val_accuracy: 0.2549 - val_loss: 1.4028 - learning_rate: 0.0010
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step - accuracy: 0.2234 - loss: 1.4501 - val_accuracy: 0.2255 - val_loss: 1.4304 - learning_rate: 0.0010
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.2870 - loss: 1.3982 - val_accuracy: 0.2843 - val_loss: 1.3909 - learning_rate: 0.0010
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step - accuracy: 0.2823 - loss: 1.3810 - val_accuracy: 0.2843 - val_loss: 1.3848 - learning_rate: 0.0010
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.2843 - loss: 1.3732 - val_accuracy: 0.2549 - val_loss: 1.4285 - learning_rate: 0.0010
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.3155 - loss: 1.3713 - val_accuracy:

- Run predictions

In [12]:
pred_labels = model.predict(X_test).argmax(axis=1)
true_labels = y_test.argmax(axis=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step


- Run Classification Report

In [13]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.17      0.18      0.17        34
           1       0.15      0.20      0.17        30
           2       0.33      0.33      0.33        39
           3       0.18      0.08      0.11        25

    accuracy                           0.21       128
   macro avg       0.20      0.20      0.20       128
weighted avg       0.21      0.21      0.21       128



## Hyperparemeter Tuning

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import log_loss
import optuna
import numpy as np
from sklearn.model_selection import KFold
def tkat_objective(trial):

    X, y = X_train, y_train

    # Suggest hyperparameters
    # lr = trial.suggest_categorical('lr', [1e-5, 1e-3, 1e-2])
    num_heads = trial.suggest_categorical('num_heads', [1, 2, 4])
    # dropout_prob = trial.suggest_categorical('dropout_prob', [0.1, 0.3, 0.5])
    hidden_units = trial.suggest_categorical('hidden_units', [64, 128, 256])
    # lstm_layers = trial.suggest_categorical('lstm_layers', [1, 2, 4])
    num_embedding = trial.suggest_categorical('classifier_units', [16, 32, 64])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])

    # Initialize the model with suggested hyperparameters
    model = TKAT(sequence_length, num_unknow_features, num_know_features, num_embedding, hidden_units, num_heads, n_ahead, use_tkan = True)
    
    model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    
    # Time series split
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):

        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        history = model.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=30, 
                    callbacks=callbacks(), 
                    shuffle=False, verbose = True)

        # Validate the model

        pred_labels = model.predict(X_val_fold)
        true_labels = y_val_fold

        # val_predictions = trainer.predict(model, val_loader)
        # val_predictions = torch.cat([x for x in val_predictions], dim=0).numpy()
        
        val_loss = CategoricalCrossentropy(from_logits=True)(true_labels, pred_labels)
        cv_scores.append(val_loss)

    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(tkat_objective, n_trials=5)

In [ ]:
# Print best hyperparameters
print("Best hyperparameters:", study.best_params)

In [ ]:
from keras.losses import CategoricalCrossentropy
final_model = TKAT(
    sequence_length, 
    num_unknow_features, 
    num_know_features, 
    study.best_params['classifier_units'], 
    study.best_params['hidden_units'], 
    study.best_params['num_heads'], 
    n_ahead, 
    use_tkan = True)

# model.compile(optimizer='adam', loss='mean_squared_error')
final_model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy']) # from_logits, allow us to automatically perform softmax activation in the loss

final_model.summary()

In [ ]:
history = final_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30, validation_split=0.2, 
                    callbacks=callbacks(), 
                    shuffle=True, verbose = True)

### Cross - Validation

In [26]:
from CPCV.cpcv import CombPurgedKFoldCVLocal
date_index = df[-y_test.shape[0]:].index

pred_times = pd.Series(date_index, index=date_index)
eval_times = pd.Series(date_index, index=date_index)

# Construct CPCV in-line with DePrado method
cpcv = CombPurgedKFoldCVLocal(
    n_splits=10,
    n_test_splits=1,
    embargo_td=pd.Timedelta(hours=0)
)

cv_split = cpcv.split(
    pd.DataFrame(X_test[:,0,:], index=date_index), 
    pd.Series(y_test[:,0], index=date_index), 
    pred_times, 
    eval_times)

In [31]:
X_train.shape

(509, 24, 3)

In [27]:
next(cv_split)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]),
 array([116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]))

In [22]:
from tkat_model_utils import cross_validate_model
cv_results = cross_validate_model(
    X_test, 
    y_test,
    model=model, 
    n_epochs=10,
    cv_split=cv_split)

Fold 1/9
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7126 - loss: 0.9060
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7180 - loss: 0.8733
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7293 - loss: 0.8387
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7363 - loss: 0.8029
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7262 - loss: 0.7665
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7366 - loss: 0.7323
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7444 - loss: 0.7100
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7126 - loss: 0.7091
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.6273 - loss: 0.7470
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.6898 - loss: 0.7730
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Cross-Validation results:
              precision    recall  f1-score   support

     Label_0       0.14      0.33      0.

In [ ]:
print(cv_results)

### MC Dropout Predictions

In [ ]:
from tkat_model_utils import mc_dropout_predictions 
# Perform MC Dropout predictions
mean_predictions, std_predictions = mc_dropout_predictions(final_model, X_test, num_iterations=100)

### Results exporting

- Export the reuslts

In [ ]:
import pandas as pd
# Save test predictions to a CSV
test_df = pd.DataFrame({
    'Prediction': predicted_labels,
    'Probability_0': [p[0] for p in mean_predictions],
    'Probability_1': [p[1] for p in mean_predictions],
    'Probability_2': [p[2] for p in mean_predictions],  # Adjust based on num_classes
    'Probability_3': [p[3] for p in mean_predictions],
    'Uncertainty_0': [u[0] for u in std_predictions],
    'Uncertainty_1': [u[1] for u in std_predictions],
    'Uncertainty_2': [u[2] for u in std_predictions],
    'Uncertainty_3': [u[3] for u in std_predictions]
})

test_df.to_csv('tkat_predictions.csv', index=False)

- Pytorch-Lightning: Save the model

In [ ]:
from torch import save
save(model.state_dict(), 'tkat_classifier.pth')